In [1]:
import os
from PIL import Image
import numpy as np

openslide_path = r'C:\Users\Donald\Documents\openslide-win64-20230414\bin'

os.environ['PATH'] = openslide_path + ";" + os.environ['PATH']
# from openslide import OpenSlide


if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(openslide_path):
        import openslide
else:
    import openslide
from openslide import OpenSlide

You need to do get crop data before running this code. I use a matlab function I made with the 2.5x or 10x tiff images.

Function is here: \\10.99.68.178\andreex\students\Donald Monkey fetus\codes\monkey\crop1024_monkey.m

In [6]:
pth = r'\\10.99.68.178\andreex\data\monkey fetus\gestational 40'
pth_mat = r'\\10.99.68.178\andreex\data\monkey fetus\gestational 40\1024tiles\new_testing_tiles_12_5\cropped_tile\matfiles'

In [25]:
# Get a list of MAT files and their base names (without extensions)
mat_files = [os.path.splitext(file)[0] for file in os.listdir(pth_mat) if file.endswith('.mat')]
slide_names = [file[:20] for file in mat_files]

# Get a list of NDPI images and their base names (without extensions)
ndpi_files = [os.path.splitext(file)[0] for file in os.listdir(pth) if file.endswith('.ndpi')]

# Find the matching files based on their base names
matching_files = set(slide_names).intersection(ndpi_files)

WSIs = sorted([filename + '.ndpi' for filename in matching_files])

crop_mats = sorted([filename + '.mat' for filename in mat_files])

In [26]:
crop_mats

['monkey_fetus_40_0600_thymus2.mat',
 'monkey_fetus_40_0718_lungs2.mat',
 'monkey_fetus_40_0754_kidney2.mat',
 'monkey_fetus_40_0768_adrenal2.mat',
 'monkey_fetus_40_0790_gonads2.mat',
 'monkey_fetus_40_0844_mesokidney2.mat',
 'monkey_fetus_40_1035_bone2.mat']

In [27]:
WSIs

['monkey_fetus_40_0600.ndpi',
 'monkey_fetus_40_0718.ndpi',
 'monkey_fetus_40_0754.ndpi',
 'monkey_fetus_40_0768.ndpi',
 'monkey_fetus_40_0790.ndpi',
 'monkey_fetus_40_0844.ndpi',
 'monkey_fetus_40_1035.ndpi']

In [29]:
res20x=0.4416
res2_5x=4

ratio = res2_5x/res20x


outpth = r'\\10.99.68.178\andreex\data\monkey fetus\gestational 40\1024tiles\new_testing_tiles_12_5\cropped_tile\20x_tiles'
if not os.path.exists(outpth):
    os.mkdir(outpth)

In [30]:
print(ratio)

9.057971014492754


In [31]:
crop_mats

['monkey_fetus_40_0600_thymus2.mat',
 'monkey_fetus_40_0718_lungs2.mat',
 'monkey_fetus_40_0754_kidney2.mat',
 'monkey_fetus_40_0768_adrenal2.mat',
 'monkey_fetus_40_0790_gonads2.mat',
 'monkey_fetus_40_0844_mesokidney2.mat',
 'monkey_fetus_40_1035_bone2.mat']

In [32]:
WSIs

['monkey_fetus_40_0600.ndpi',
 'monkey_fetus_40_0718.ndpi',
 'monkey_fetus_40_0754.ndpi',
 'monkey_fetus_40_0768.ndpi',
 'monkey_fetus_40_0790.ndpi',
 'monkey_fetus_40_0844.ndpi',
 'monkey_fetus_40_1035.ndpi']

In [34]:
from scipy.io import loadmat

file_format = '.tif'

for i, file in enumerate(crop_mats):
    mat_file_name = os.path.join(pth_mat, file)

    WSI_path = os.path.join(pth, WSIs[i])

    f_out_name = f'{os.path.join(os.path.join(outpth,crop_mats[i][:-4]))}{file_format}'

    if os.path.exists(f_out_name):
        print(f'Skipping cropping for {WSIs[i]} ({i+1}/{len(crop_mats)})... Already exists')

    else:
        print(f'Cropping {WSIs[i]} ({i+1}/{len(crop_mats)})')

        # print(mat_file_name)
        # print(WSI_path)

        WSI_slide = openslide.OpenSlide(WSI_path)

        wsi_width, wsi_height = WSI_slide.level_dimensions[0]

        data = loadmat(mat_file_name)

        # crop_width = int(data['crop_width']*ratio)
        # crop_height = int(data['crop_height']*ratio)


        crop_width = 256
        crop_height = 256
        crop_x = int(data['crop_x']*ratio)
        crop_y = int(data['crop_y']*ratio)

        lower_x = crop_x
        upper_y = crop_y
        upper_x = crop_x + crop_width
        lower_y = crop_y - crop_height

        print(crop_x)
        print(crop_y)
        print(crop_width)
        print(crop_height)

        region = WSI_slide.read_region((crop_x, crop_y), 0, (crop_width, crop_height)) # level = 0 in middle
        region_image = Image.new('RGB', region.size)
        region_image.paste(region, (0, 0))

        region_image.save(f_out_name)
        WSI_slide.close()


Cropping monkey_fetus_40_0600.ndpi (1/7)
30330
6799
256
256
Cropping monkey_fetus_40_0718.ndpi (2/7)
24547
6349
256
256
Cropping monkey_fetus_40_0754.ndpi (3/7)
28000
7211
256
256
Cropping monkey_fetus_40_0768.ndpi (4/7)
26761
7151
256
256
Cropping monkey_fetus_40_0790.ndpi (5/7)
27015
8310
256
256
Cropping monkey_fetus_40_0844.ndpi (6/7)
26512
6974
256
256
Cropping monkey_fetus_40_1035.ndpi (7/7)
23170
9438
256
256


In [38]:
crop_x


19818

In [39]:
crop_y

20134